## Import libraries

In [1]:
import cdsapi
import zipfile
import xarray as xr 
import rioxarray as rio 
from tqdm import tqdm
import pandas as pd
from datetime import datetime as dt
import datetime
import rasterio
from rasterio.plot import show
import numpy as np
import schedule
import netCDF4 as nc
from netCDF4 import Dataset
import os, sys
import copy
import platform
import tempfile
import logging
from math import log10, cos, sin, asin, sqrt, exp, pi, radians
from collections import namedtuple
from bisect import bisect_left
import textwrap
import sqlite3
from collections import Iterable
import pathlib as Path
import glob
from osgeo import gdal

/tmp/ipykernel_8664/3008149916.py:25: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


# Define functions

In [2]:
def download_AgERA5_month(selected_area, variables, query_date):

    print("===== Downloading {} for {} =====".format(variables, query_date))

    try:
        query_year = query_date
        c = cdsapi.Client()

        if not os.path.exists('data/0_downloads/'):
            os.makedirs('data/0_downloads/')

        for variable in variables :
            
            print("\nDownloading values for variable",variable,"/",query_year)

            zip_path = 'data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'

            request = {
                    'format': 'zip',
                    'day': [
                        '01', '02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                        '13', '14', '15',
                        '16', '17', '18',
                        '19', '20', '21',
                        '22', '23', '24',
 a                       '25', '26', '27',
                        '28', '29', '30',
                        '31',
                    ],
                    'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11','12'
                    ],
                    'year': query_year,
                    'variable': variable[0],
                    'statistic': variable[1],
                    'area': area[selected_area],
                }

            # la requête doit être adaptée pour cette variable
            if variable[0] == "solar_radiation_flux" :
                del request["statistic"]

            if variable[0] == "precipitation_flux" :
                del request["statistic"]
                
                
            c.retrieve(
                'sis-agrometeorological-indicators',
                request,
                zip_path)

            print("Download OK")

    except :
        print("/!\ Download NOT OK")

In [3]:
def extract_agERA5_month(selected_area, variables, query_date):

    print("===== extract_agERA5_month =====")

    try:
        query_year = query_date

        for variable in tqdm(variables) :

            zip_path = 'data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'
            extraction_path = 'data/1_extraction/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'/'

            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extraction_path)
            except:
                pass
            
            print("Extraction OK")
            
            print('Removing folder:', zip_path)
            os.remove(zip_path)

    except :
        print("/!\ Extraction NOT OK")

In [4]:
Celsius2Kelvin = lambda x: x + 273.16
hPa2kPa = lambda x: x/10.

# Saturated Vapour pressure [kPa] at temperature temp [C]
SatVapourPressure = lambda temp: 0.6108 * exp((17.27 * temp) / (237.3 + temp))

# Named tuple for returning results of ASTRO
astro_nt = namedtuple("AstroResults", "DAYL, DAYLP, SINLD, COSLD, DIFPP, "
                                      "ATMTR, DSINBE, ANGOT")
def astro(day, latitude, radiation, _cache={}):
    """python version of ASTRO routine by Daniel van Kraalingen.
    
    This subroutine calculates astronomic daylength, diurnal radiation
    characteristics such as the atmospheric transmission, diffuse radiation etc.
    :param day:         date/datetime object
    :param latitude:    latitude of location
    :param radiation:   daily global incoming radiation (J/m2/day)
    output is a `namedtuple` in the following order and tags::
        DAYL      Astronomical daylength (base = 0 degrees)     h      
        DAYLP     Astronomical daylength (base =-4 degrees)     h      
        SINLD     Seasonal offset of sine of solar height       -      
        COSLD     Amplitude of sine of solar height             -      
        DIFPP     Diffuse irradiation perpendicular to
                  direction of light                         J m-2 s-1 
        ATMTR     Daily atmospheric transmission                -      
        DSINBE    Daily total of effective solar height         s
        ANGOT     Angot radiation at top of atmosphere       J m-2 d-1
 
    Authors: Daniel van Kraalingen
    Date   : April 1991
 
    Python version
    Author      : Allard de Wit
    Date        : January 2011
    """

    # Check for range of latitude
    if abs(latitude) > 90.:
        msg = "Latitude not between -90 and 90"
        raise RuntimeError(msg)
    LAT = latitude
        
    # Determine day-of-year (IDAY) from day
    IDAY = doy(day)
    
    # reassign radiation
    AVRAD = radiation

    # Test if variables for given (day, latitude, radiation) were already calculated
    # in a previous run. If not (e.g. KeyError) calculate the variables, store
    # in cache and return the value.
    try:
        return _cache[(IDAY, LAT, AVRAD)]
    except KeyError:
        pass

    # constants
    RAD = radians(1.)
    ANGLE = -4.

    # Declination and solar constant for this day
    DEC = -asin(sin(23.45*RAD)*cos(2.*pi*(float(IDAY)+10.)/365.))
    SC  = 1370.*(1.+0.033*cos(2.*pi*float(IDAY)/365.))

    # calculation of daylength from intermediate variables
    # SINLD, COSLD and AOB
    SINLD = sin(RAD*LAT)*sin(DEC)
    COSLD = cos(RAD*LAT)*cos(DEC)
    AOB = SINLD/COSLD

    # For very high latitudes and days in summer and winter a limit is
    # inserted to avoid math errors when daylength reaches 24 hours in 
    # summer or 0 hours in winter.

    # Calculate solution for base=0 degrees
    if abs(AOB) <= 1.0:
        DAYL  = 12.0*(1.+2.*asin(AOB)/pi)
        # integrals of sine of solar height
        DSINB  = 3600.*(DAYL*SINLD+24.*COSLD*sqrt(1.-AOB**2)/pi)
        DSINBE = 3600.*(DAYL*(SINLD+0.4*(SINLD**2+COSLD**2*0.5))+
                 12.*COSLD*(2.+3.*0.4*SINLD)*sqrt(1.-AOB**2)/pi)
    else:
        if AOB >  1.0: DAYL = 24.0
        if AOB < -1.0: DAYL = 0.0
        # integrals of sine of solar height	
        DSINB = 3600.*(DAYL*SINLD)
        DSINBE = 3600.*(DAYL*(SINLD+0.4*(SINLD**2+COSLD**2*0.5)))

    # Calculate solution for base=-4 (ANGLE) degrees
    AOB_CORR = (-sin(ANGLE*RAD)+SINLD)/COSLD
    if abs(AOB_CORR) <= 1.0:
        DAYLP = 12.0*(1.+2.*asin(AOB_CORR)/pi)
    elif AOB_CORR > 1.0:
        DAYLP = 24.0
    elif AOB_CORR < -1.0:
        DAYLP = 0.0

    # extraterrestrial radiation and atmospheric transmission
    ANGOT = SC*DSINB
    # Check for DAYL=0 as in that case the angot radiation is 0 as well
    if DAYL > 0.0:
        ATMTR = AVRAD/ANGOT
    else:
        ATMTR = 0.

    # estimate fraction diffuse irradiation
    if (ATMTR > 0.75):
        FRDIF = 0.23
    elif (ATMTR <= 0.75) and (ATMTR > 0.35):
        FRDIF = 1.33-1.46*ATMTR
    elif (ATMTR <= 0.35) and (ATMTR > 0.07):
        FRDIF = 1.-2.3*(ATMTR-0.07)**2
    else:  # ATMTR <= 0.07
        FRDIF = 1.

    DIFPP = FRDIF*ATMTR*0.5*SC

    retvalue = astro_nt(DAYL, DAYLP, SINLD, COSLD, DIFPP, ATMTR, DSINBE, ANGOT)
    _cache[(IDAY, LAT, AVRAD)] = retvalue

    return retvalue

def doy(day):
    """Converts a date or datetime object to day-of-year (Jan 1st = doy 1)
    """
    # Check if day is a date or datetime object
    if isinstance(day, (datetime.date, datetime.datetime)):
        return day.timetuple().tm_yday
    else:
        msg = "Parameter day is not a date or datetime object."
        raise RuntimeError(msg)

def penman_monteith(DAY, LAT, ELEV, TMIN, TMAX, AVRAD, VAP, WIND2):
    """Calculates reference ET0 based on the Penman-Monteith model.
     This routine calculates the potential evapotranspiration rate from
     a reference crop canopy (ET0) in mm/d. For these calculations the
     analysis by FAO is followed as laid down in the FAO publication
     `Guidelines for computing crop water requirements - FAO Irrigation
     and drainage paper 56 <http://www.fao.org/docrep/X0490E/x0490e00.htm#Contents>`_
    Input variables::
        DAY   -  Python datetime.date object                   -
        LAT   -  Latitude of the site                        degrees
        ELEV  - Elevation above sea level                      m
        TMIN  - Minimum temperature                            C
        TMAX  - Maximum temperature                            C
        AVRAD - Daily shortwave radiation                   J m-2 d-1
        VAP   - 24 hour average vapour pressure               hPa
        WIND2 - 24 hour average windspeed at 2 meter          m/s
    Output is:
        ET0   - Penman-Monteith potential transpiration
                rate from a crop canopy                     [mm/d]
    """

    # psychrometric instrument constant (kPa/Celsius)
    PSYCON = 0.665
    # albedo and surface resistance [sec/m] for the reference crop canopy
    REFCFC = 0.23; CRES = 70.
    # latent heat of evaporation of water [J/kg == J/mm] and
    LHVAP = 2.45E6
    # Stefan Boltzmann constant (J/m2/d/K4, e.g multiplied by 24*60*60)
    STBC = 4.903E-3
    # Soil heat flux [J/m2/day] explicitly set to zero
    G = 0.

    # mean daily temperature (Celsius)
    TMPA = (TMIN+TMAX)/2.

    # Vapour pressure to kPa
    VAP = hPa2kPa(VAP)

    # atmospheric pressure at standard temperature of 293K (kPa)
    T = 293.0
    PATM = 101.3 * pow((T - (0.0065*ELEV))/T, 5.26)

    # psychrometric constant (kPa/Celsius)
    GAMMA = PSYCON * PATM * 1.0E-3

    # Derivative of SVAP with respect to mean temperature, i.e.
    # slope of the SVAP-temperature curve (kPa/Celsius);
    SVAP_TMPA = SatVapourPressure(TMPA)
    DELTA = (4098. * SVAP_TMPA)/pow((TMPA + 237.3), 2)

    # Daily average saturated vapour pressure [kPa] from min/max temperature
    SVAP_TMAX = SatVapourPressure(TMAX)
    SVAP_TMIN = SatVapourPressure(TMIN)
    SVAP = (SVAP_TMAX + SVAP_TMIN) / 2.

    # measured vapour pressure not to exceed saturated vapour pressure
    VAP = min(VAP, SVAP)

    # Longwave radiation according at Tmax, Tmin (J/m2/d)
    # and preliminary net outgoing long-wave radiation (J/m2/d)
    STB_TMAX = STBC * pow(Celsius2Kelvin(TMAX), 4)
    STB_TMIN = STBC * pow(Celsius2Kelvin(TMIN), 4)
    RNL_TMP = ((STB_TMAX + STB_TMIN) / 2.) * (0.34 - 0.14 * sqrt(VAP))

    # Clear Sky radiation [J/m2/DAY] from Angot TOA radiation
    # the latter is found through a call to astro()
    r = astro(DAY, LAT, AVRAD)
    CSKYRAD = (0.75 + (2e-05 * ELEV)) * r.ANGOT

    if CSKYRAD > 0:
        # Final net outgoing longwave radiation [J/m2/day]
        RNL = RNL_TMP * (1.35 * (AVRAD/CSKYRAD) - 0.35)

        # radiative evaporation equivalent for the reference surface
        # [mm/DAY]
        RN = ((1-REFCFC) * AVRAD - RNL)/LHVAP

        # aerodynamic evaporation equivalent [mm/day]
        EA = ((900./(TMPA + 273)) * WIND2 * (SVAP - VAP))

        # Modified psychometric constant (gamma*)[kPa/C]
        MGAMMA = GAMMA * (1. + (CRES/208.*WIND2))

        # Reference ET in mm/day
        ET0 = (DELTA * (RN-G))/(DELTA + MGAMMA) + (GAMMA * EA)/(DELTA + MGAMMA)
        ET0 = max(0., ET0)
    else:
        ET0 = 0.

    return ET0


# Download and extract

In [5]:
first = pd.date_range(start='1979', end='2022', freq='YS')
datelist = [i.strftime('%Y') for i in first]

for year in datelist:
    query_date = year
    area = {'Swaziland': [-27.46,30.28,-25.64,32.46]}
    selected_area = "Swaziland"
    variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily"),
    ("solar_radiation_flux", "daily"),
    ("vapour_pressure", "24_hour_mean"),
    ("10m_wind_speed", "24_hour_mean")]
    
    download_AgERA5_month(selected_area, variables, query_date)
    print('Finished downloading:',year)
    extract_agERA5_month(selected_area, variables, query_date)
    print('Finished extracting')

===== Downloading [('2m_temperature', '24_hour_minimum'), ('2m_temperature', '24_hour_maximum'), ('precipitation_flux', 'daily'), ('solar_radiation_flux', 'daily'), ('vapour_pressure', '24_hour_mean'), ('10m_wind_speed', '24_hour_mean')] for 2021 =====



/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-23 05:56:35,510 INFO Welcome to the CDS
2022-10-23 05:56:35,511 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-23 05:56:35,840 INFO Request is completed
2022-10-23 05:56:35,841 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-comp

Download OK



/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-23 05:56:39,801 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/dataset-sis-agrometeorological-indicators-5eb13f2c-d7d2-4bd9-8b64-3a9eff760d15.zip to data/0_downloads/AgERA5_Swaziland_2m_temperature_24_hour_maximum_2021.zip (7.6M)
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download-0015-clone.copernicus-climate.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-23 05:56:41,462 INF

Download OK



...

Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_2m_temperature_24_hour_minimum_2022.zip
Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_2m_temperature_24_hour_maximum_2022.zip


 67%|██████████████████████████████████████████████████████████████▋                               | 4/6 [00:00<00:00,  7.69it/s]

Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_precipitation_flux_daily_2022.zip
Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_solar_radiation_flux_daily_2022.zip


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  7.32it/s]

Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_vapour_pressure_24_hour_mean_2022.zip
Extraction OK
Removing folder: data/0_downloads/AgERA5_Swaziland_10m_wind_speed_24_hour_mean_2022.zip
Finished extracting


# Aggregate netCDF

### Daily files

In [22]:
variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily"),
    ("solar_radiation_flux", "daily"),
    ("vapour_pressure", "24_hour_mean"),
    ("10m_wind_speed", "24_hour_mean")]
selected_area = "Swaziland"

first = pd.date_range(start='1979', end='2022', freq='YS')
datelist = [i.strftime('%Y') for i in first]

for year in datelist:
    for variable in variables:      
        files = glob.glob(os.path.join('data','1_extraction','*{}_{}_{}'.format(variable[0],variable[1],year),'*'))
        df = xr.open_mfdataset(files, parallel=True, engine="netcdf4", decode_times=True, chunks={'time': 300},
                       data_vars='minimal', coords='minimal', compat='override')
        if not os.path.exists('data/2_aggregate/'):
            os.makedirs('data/2_aggregate/')
        print('Writing {} {}'.format(variable,year))
        df.to_netcdf('data/2_aggregate/AgERA5_{}_{}_{}_{}.nc'.format(selected_area,variable[0],variable[1],year), mode='w')
        df.close()
        print('Finished writing {} {}'.format(variable, year))

Writing ('2m_temperature', '24_hour_minimum') 1979
Finished writing ('2m_temperature', '24_hour_minimum') 1979
Writing ('2m_temperature', '24_hour_maximum') 1979
Finished writing ('2m_temperature', '24_hour_maximum') 1979
Writing ('precipitation_flux', 'daily') 1979
Finished writing ('precipitation_flux', 'daily') 1979
Writing ('solar_radiation_flux', 'daily') 1979
Finished writing ('solar_radiation_flux', 'daily') 1979
Writing ('vapour_pressure', '24_hour_mean') 1979
Finished writing ('vapour_pressure', '24_hour_mean') 1979
Writing ('10m_wind_speed', '24_hour_mean') 1979
Finished writing ('10m_wind_speed', '24_hour_mean') 1979
Writing ('2m_temperature', '24_hour_minimum') 1980
Finished writing ('2m_temperature', '24_hour_minimum') 1980
Writing ('2m_temperature', '24_hour_maximum') 1980
Finished writing ('2m_temperature', '24_hour_maximum') 1980
Writing ('precipitation_flux', 'daily') 1980
Finished writing ('precipitation_flux', 'daily') 1980
Writing ('solar_radiation_flux', 'daily') 1

### Annual files

In [26]:
variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily"),
    ("solar_radiation_flux", "daily"),
    ("vapour_pressure", "24_hour_mean"),
    ("10m_wind_speed", "24_hour_mean")]

selected_area = "Swaziland"

for variable in variables:      
    files = glob.glob(os.path.join('data/2_aggregate','*{}_{}*'.format(variable[0],variable[1])))
    df = xr.open_mfdataset(files, parallel=True, engine="netcdf4", decode_times=True, chunks={'time': 1000},
                       data_vars='minimal', coords='minimal', compat='override')
    if not os.path.exists('data/3_combined'):
        os.makedirs('data/3_combined')
    print('Writing {}'.format(variable))
    df.to_netcdf('data/3_combined/AgERA5_{}_{}_{}.nc'.format(selected_area,variable[0],variable[1]), mode='w')
    df.close()
    print('Finished writing {}'.format(variable))

Writing ('2m_temperature', '24_hour_minimum')
Finished writing ('2m_temperature', '24_hour_minimum')
Writing ('2m_temperature', '24_hour_maximum')
Finished writing ('2m_temperature', '24_hour_maximum')
Writing ('precipitation_flux', 'daily')
Finished writing ('precipitation_flux', 'daily')
Writing ('solar_radiation_flux', 'daily')
Finished writing ('solar_radiation_flux', 'daily')
Writing ('vapour_pressure', '24_hour_mean')
Finished writing ('vapour_pressure', '24_hour_mean')
Writing ('10m_wind_speed', '24_hour_mean')
Finished writing ('10m_wind_speed', '24_hour_mean')


# Calculate ETo and export for sites

In [5]:
DataDir = 'data/3_combined/'

SiteFile = pd.DataFrame()

Sites = [('Big_Bend','-26.8','31.9','170')]

Kelvin2Celsius = lambda x: x - 273.16

variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily"),
    ("solar_radiation_flux", "daily"),
    ("vapour_pressure", "24_hour_mean"),
    ("10m_wind_speed", "24_hour_mean")]

for site in Sites:
    sitename = site[0]
    Latitude = site[1]
    Longitude = site[2]
    Elevation = site[3]
    print('Getting data for:',sitename)
    for variable in variables:
        files = glob.glob(os.path.join(DataDir,'*{}_{}*'.format(variable[0],variable[1])))
        df = xr.open_mfdataset(files, parallel=True, engine="netcdf4")
        pnt = df.sel(lat=Latitude,lon=Longitude,  method="nearest")
        df = pnt.to_dataframe()
        df = df.drop(columns={'lat','lon'})
        df.reset_index(inplace=True)
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        
        if len(SiteFile) == 0:
            SiteFile = df
        else:
            SiteFile = pd.concat([df,SiteFile],axis=1)
    
    SiteFile = SiteFile.rename(columns={'Wind_Speed_10m_Mean':'WIND2', 'Vapour_Pressure_Mean':'VAP','Solar_Radiation_Flux':'AVRAD',
       'Precipitation_Flux':'Precipitation', 'Temperature_Air_2m_Max_24h':'MaxTemp',
       'Temperature_Air_2m_Min_24h':'MinTemp'})
    
    SiteFile['LAT'] = float(Latitude)
    SiteFile['ELEV'] = float(Elevation)
    SiteFile.index.names = ['Date']
    SiteFile[['MinTemp','MaxTemp']] = Kelvin2Celsius(SiteFile[['MinTemp','MaxTemp']])
    
    SiteFile.reset_index(inplace=True)
    
    print('Calculating Evapotranspiration')
    for idx, row in SiteFile.iterrows():
        SiteFile.loc[idx,'ReferenceET'] = penman_monteith(DAY=row['Date'], LAT=row['LAT'], 
                    ELEV=row['ELEV'],TMIN=row['MinTemp'],
                    TMAX=row['MaxTemp'],AVRAD=row['AVRAD'],
                    VAP=row['VAP'],WIND2=row['WIND2'])
    
    if not os.path.isdir('data/04_sitefiles'):
        os.makedirs('data/04_sitefiles')
    
    exportfile = SiteFile[["MinTemp","MaxTemp","Precipitation","ReferenceET","Date"]]
    print("Saving csv file for:", sitename)
    exportfile.to_csv('data/04_sitefiles/AgERA5_{}.csv'.format(sitename))

Getting data for: Big_Bend


/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.g

Calculating Evapotranspiration
Saving csv file for: Big_Bend
